# Custom agent
---

En este cuaderno explica cómo crear su propio agente personalizado.

Un agente consiste en dos partes:

- Tools: Las herramientas (tools) que el agente tiene disponibles para usar.
- The agent class: decide qué acción tomar.



En este cuaderno explicamos cómo crear un agente personalizado.

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper

load_dotenv(find_dotenv())

True

Generaremos una herramienta (tool) que nos permita hacer busquedas de elementos actuales, usaremos SerpAPIWrapper

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need answer questions about current events",
        return_direct=True,
    )
]

In [3]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish

class FakeAgent(BaseSingleActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]
    
    def plan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]], 
        **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given tinput, decide what to do.
        
        Args:
            intermediate_steps: Steps the LLM has taken to date, along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")
    
    async def aplan(
        self,
        intermediate_steps: List[Tuple[AgentAction, str]], 
        **kwargs: Any       
    ) -> Union[AgentAction, AgentFinish]:
        """Given tinput, decide what to do.
        
        Args:
            intermediate_steps: Steps the LLM has taken to date, along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")

In [4]:
agent = FakeAgent()

In [5]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [8]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
38,781,291 people


> Finished chain.


'38,781,291 people'